# Logistic Regression

- <https://towardsdatascience.com/logistic-regression-from-scratch-69db4f587e17>
- <https://aihubprojects.com/logistic-regression-from-scratch/>
- <https://thelaziestprogrammer.com/sharrington/math-of-machine-learning/solving-logreg-newtons-method>
- <https://www.pyimagesearch.com/2016/10/17/stochastic-gradient-descent-sgd-with-python/>

In [ ]:
import matplotlib.pyplot as plot
from sklearn.linear_model import LogisticRegression as ScikitLogisticRegression

from utility import LogisticRegression, Methods, display, load_dataset, split_dataset

The following class contains model fitting and prediction methods using Logistic Regression
using [Gradient Descent](https://en.wikipedia.org/wiki/Gradient_descent) with [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) and [Log Loss](https://en.wikipedia.org/wiki/Log_loss) function.

- Log Loss (`log_loss`): $$L_{\log}(y, p) = -(y \log (p) + (1 - y) \log (1 - p))$$
- Sum of Squared Errors (SSE) (`sse_loss`): $$SSE(y, p) = \sum_{i}^{n} (p - y)^2$$

load training and test data from CSV file

In [ ]:
dataset, target = load_dataset("data/forestfires.csv")
train_x, train_y, test_x, test_y = split_dataset(dataset, target, 0.75)
print(f"Training set size: {len(train_x)}, Testing set size: {len(test_x)}")

train the model with training data using Gradient Descent and plot loss during iteration and plot confusion matrix.

In [ ]:
model = LogisticRegression()
loss_plot = plot.plot(model.fit(train_x, train_y, Methods.GD))
predictions = model.predict(test_x)
display(test_y, predictions)

train the model with training data using Stochastic Gradient Descent and plot loss during iteration and plot confusion matrix.

In [ ]:
model = LogisticRegression()
loss_plot = plot.plot(model.fit(train_x, train_y, Methods.SGD))
predictions = model.predict(test_x)
display(test_y, predictions)

Use scikit-learn [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
model = ScikitLogisticRegression()
model.fit(train_x, train_y)
predictions = model.predict(test_x)
display(test_y, predictions)